# Deploy Machine Learning (scikit-learn) Models in IBM Cloud - Watson Studio

IBM Watson Studio provides tools for data scientists, application developers and subject matter experts to collaboratively and easily work with data to build and train models at scale. It gives you the flexibility to build models where your data resides and deploy anywhere in a hybrid environment so you can operationalize data science faster. IBM Watson Studio provides various tools for designing, training, and managing machine learning models:

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Insert your dataset here


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [26]:
#Check Missing Values
dataset.isnull().any()

User ID            False
Gender             False
Age                False
EstimatedSalary    False
Purchased          False
dtype: bool

In [ ]:
#Spilt Dependent and Independent Variables
X = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state = 0)

In [ ]:
# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
#Finding the accuracy score
from sklearn.metrics import accuracy_score
print("Accuracy Score: ",accuracy_score(y_test,y_pred)*100,"%")

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2019-06-20 11:40:56,006 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2019-06-20 11:45:53,169 - watson_machine_learning_client.wml_client_error - WARNING - Deployment creation failed. Error: 402. {"trace":"81de7e6aeb1f8ae6460ec0fd62f0f76d","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}
2019-06-20 11:51:35,144 - watson_machine_learning_client.wml_client_error - WARNING - Failure during scoring. (POST https://us-south.

In [7]:
wml={
    
}

In [8]:
client = WatsonMachineLearningAPIClient(wml)

In [9]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "", 
               client.repository.ModelMetaNames.NAME: "MyModel"}

In [11]:
model_artifact =client.repository.store_model(classifier, meta_props=model_props)


In [ ]:
client.repository.list()

In [20]:
published_model_uid = client.repository.get_model_uid(model_artifact)
created_deployment = client.deployments.create(published_model_uid, name="MyDeployment")



#######################################################################################

Synchronous deployment creation for uid: 'bcc4e489-fc27-4fea-837c-449ae5958860' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8fc64951-41ea-4c38-b345-c91ae311b69c'
------------------------------------------------------------------------------------------------




In [22]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)


# In[105]:


print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/849e0c17-8992-43a4-8f4e-700687b1aa49/deployments/8fc64951-41ea-4c38-b345-c91ae311b69c/online


In [25]:
scoring_payload = {"fields": ["Age","Salary"],"values": [[25,50000]]}
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print(predictions)

{'fields': ['prediction', 'probability'], 'values': [[0, [1.0, 0.0]]]}

In [16]:
client.deployments.list("de8eebf1-7c57-429d-9831-21c5fd4912a3")

------------------------------------  ------------  ------  --------------  ------------------------  --------------  -------------
GUID                                  NAME          TYPE    STATE           CREATED                   FRAMEWORK       ARTIFACT TYPE
de8eebf1-7c57-429d-9831-21c5fd4912a3  deployment    online  DEPLOY_FAILURE  2019-05-15T16:48:07.746Z  tensorflow-1.5  model
42e95127-1749-4101-a279-9276ebce0c73  deployment    online  DEPLOY_FAILURE  2019-05-15T16:46:36.233Z  tensorflow-1.5  model
7f78780a-5deb-4aaf-9d4b-a68354559b13  deployment    online  DEPLOY_SUCCESS  2019-04-18T10:57:15.882Z  tensorflow-1.5  model
8b3dcf6f-eefc-40fe-81e1-b08fb260fc18  deployment    online  DEPLOY_FAILURE  2019-04-18T09:36:48.813Z  tensorflow-1.5  model
404c0782-6d6e-4490-a4fb-0419772dfa82  MyDeployment  online  DEPLOY_SUCCESS  2019-02-15T13:57:20.687Z  wml-1.1         model
------------------------------------  ------------  ------  --------------  ------------------------  --------------